This script setup the experiments, assuming that the l4s part has been installed correctly. 

In [1]:
import subprocess

tx0_prefix = "ssh PeterYao@pc490.emulab.net"
router0_prefix = "ssh PeterYao@pc500.emulab.net"
router1_prefix = "ssh PeterYao@pc487.emulab.net"
rx0_prefix = "ssh PeterYao@pc816.emulab.net"

nodes_prefix = [tx0_prefix, router0_prefix, router1_prefix, rx0_prefix]

class node:
    def __init__(self, node_ssh_prefix) -> None:
        self.ssh_prefix = node_ssh_prefix

    def execute(self, command, background=False):
        if background:
            print("executing in background")
            # full_command = f"{self.ssh_prefix} 'setsid nohup {command} > /dev/null 2>&1 &'"
            full_command = f'{self.ssh_prefix} "{command}"'
            subprocess.Popen(full_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        else:
            full_command = f'{self.ssh_prefix} "{command}"'
            result = subprocess.run(full_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            if result.returncode == 0:
                print(result.stdout.decode('utf-8'))
            else:
                print(f"Error: {result.stderr.decode('utf-8')}")
        return None
        
tx0_node = node(tx0_prefix)
delay_node = node(router0_prefix)
router_node = node(router1_prefix)
rx0_node = node(rx0_prefix)

nodes = [tx0_node, delay_node, router_node, rx0_node]





In [2]:
from fabric import Connection


tx = Connection(
    host='pc490.emulab.net',
    user = 'PeterYao',
    port=22,
)




delay = Connection(
    host='pc500.emulab.net',
    user = 'PeterYao',
    port=22,
)


router = Connection(
    host='pc487.emulab.net',
    user='PeterYao',
    port=22,
)

rx = Connection(
    host='pc816.emulab.net',
    user
    = 'PeterYao',  
    port=22,
)

conns = [router, delay, tx, rx]


# Experiment 3  add l4s


### Install the l4s kernel in each node. 

In [ ]:
rx.put("install_l4s_kernel.sh")

In [ ]:
rx.run(" cp -r /mydata/core-network-5g/etc/ /local/repository/")

In [ ]:


rx.run("sudo ip route add 12.1.1.64/26 via 192.168.70.140")
rx.run("sudo ip route add 12.1.1.128/25 via 192.168.70.134")

rx.run("sudo iptables -P FORWARD ACCEPT ")

at the ue1 namespace:
```
sudo ip netns exec ue1 bash
sudo ip route add default via 10.201.1.100
```
so that the icmp messages can be routed back to the sender. 


at the ue3 namesapce:
```
sudo ip netns exec ue3 bash
sudo ip route add default via 10.203.1.100
```

In [ ]:
# delay.put("install_l4s_kernel.sh")
# tx.put("install_l4s_kernel.sh")
# router.put("install_l4s_kernel.sh")

delay.run("chmod +x install_l4s_kernel.sh")
tx.run("chmod +x install_l4s_kernel.sh")
router.run("chmod +x install_l4s_kernel.sh")

delay.run("bash ./install_l4s_kernel.sh", Warning=True)
tx.run("bash ./install_l4s_kernel.sh", Warning=True)
router.run("bash ./install_l4s_kernel.sh", Warning=True)


In [ ]:
for c in conns:
    c.run("uname -r")

In [ ]:
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git
cd iproute2
sudo chmod +x configure
sudo ./configure
sudo make
sudo make install"""

router.run(cmd_dualpi2)

router.run("sudo modprobe sch_dualpi2")

In [ ]:
commands_accecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=prague  
sudo sysctl -w net.ipv4.tcp_ecn=3''' 

rx.run(commands_accecn)
tx.run(commands_accecn)


In [ ]:
# commands_noecn = "bash -c 'sudo sysctl -w net.ipv4.tcp_congestion_control=cubic; sudo sysctl -w net.ipv4.tcp_ecn=0'"
# for node in nodes:
#     node.execute(commands_noecn)
    
print("validating...")
for node in nodes:
    node.execute("sudo sysctl net.ipv4.tcp_congestion_control")
    node.execute("sudo sysctl net.ipv4.tcp_ecn")

In [ ]:
# for some reason, this cannot be automated

for node in nodes:
    # Download and unzip the kernel package
    node.execute("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.execute("sudo apt install unzip")
    node.execute("unzip l4s-testing.zip")
    
    # Install the kernel packages and update GRUB
    node.execute("sudo dpkg --install debian_build/*")
    node.execute("sudo update-grub")
    node.execute("sudo reboot")

for node in nodes:
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git
cd iproute2
sudo chmod +x configure
sudo ./configure
sudo make
sudo make install"""

router.run(cmd_dualpi2)
router.run("sudo modprobe sch_dualpi2")

In [ ]:
packages = ['iperf3', 'net-tools', 'moreutils']
for conn in conns:
    for package in packages:
        conn.sudo(f'sudo apt update; apt-get -y install {package}')

In [16]:
# modify the delay on the delay node
base_rtt = 25
delay_interfaces = ["enp5s0f0", "eno3"]

for e in delay_interfaces:
    cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=base_rtt/2)
    delay.run(cmds)

In [ ]:
delay.run("sudo tc qdisc show dev enp5s0f0")
delay.run("sudo tc qdisc show dev eno3")

In [18]:
# setup the router queueing discipline
router_egress_name = "eno3"


In [ ]:
# set up the btl node
n_bdp = 2
base_rtt = 25
btl_capacity = 100 #in Mbps

# fixed values
btl_limit    = int(1000*n_bdp*btl_capacity*base_rtt/8) # limit of the bottleneck, n_bdp x BDP in bytes 
packet_number=int(btl_limit/1500)+1

print("btl limit: ", btl_limit)
print("packet number: ", packet_number)

In [ ]:

cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            '''.format(iface=router_egress_name, capacity=btl_capacity, buffer=btl_limit)
            
cmds_specific = "sudo tc qdisc add dev {iface} parent 1:3 handle 3: dualpi2 target {threshold}ms".format(iface=router_egress_name, threshold=5)

router.run(cmds_prefix)    
router.run(cmds_specific)
router.run("sudo tc qdisc show dev {iface}".format(iface=router_egress_name))  




In [ ]:
router.run("sudo tc qdisc show dev {iface}".format(iface=router_egress_name))  


In [ ]:
for c in conns:
    c.run("sudo sysctl -w net.ipv4.tcp_no_metrics_save=1")

### in UE1 namespace
```bash
sudo sysctl -w  net.ipv4.tcp_congestion_control=prague
sudo sysctl -w  net.ipv4.tcp_ecn=3
```

In [ ]:
# run the iperf command

rx.sudo("killall iperf3", warn=True)

# router_egress_name = "eno3"
# router.run("sudo ethtool -S {iface}".format(iface=router_egress_name))
# router.run("ip -s link show {iface}".format(iface=router_egress_name))

rx.sudo("ip netns exec ue1 iperf3 -s -1 -p 4008 -D")
rx.sudo("ip netns exec ue3 iperf3 -s -1 -p 4008 -D")

local_file_path = r"d:\5g notes\5G-E2E-Wireless-Notes-OAI\exp-9-15\exp.sh"

router.run("chmod +x ~/monitor_dual.sh")

# the monitor queue length shell script has already been copied to the router1 node
router.run("nohup ./monitor.sh eno3 60 1 > monitor.log 2>&1 &", pty=False)

# the put command is funny on windows, so I copy paster the exp file manually to the tx node
tx.run("chmod +x ~/exp.sh")
tx.run("~/exp.sh prague")



In [ ]:
tx.close()
tx = Connection(
    host='pc490.emulab.net',
    user = 'PeterYao',
    port=22,
)
tx.get("prague-result-ue1.json")
tx.get("prague-result-ue2.json")
tx.get("prague-ss-ue1.txt")
tx.get("prague-ss-ue2.txt")

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# Replace these with the paths to your actual iperf JSON result files
file1 = 'prague-result-ue1.json'
file2 = 'prague-result-ue2.json'

def load_iperf_data(filename):
    """Load iperf JSON data from a file and return a pandas DataFrame."""
    with open(filename, 'r') as f:
        data = json.load(f)
    
    # Initialize lists to store the extracted data
    intervals = data['intervals']
    start_times = []
    end_times = []
    durations = []
    throughputs = []
    retransmissions = []
    
    # Iterate over each interval in the data
    for interval in intervals:
        sum_data = interval['sum']
        start_times.append(sum_data['start'])
        end_times.append(sum_data['end'])
        durations.append(sum_data['seconds'])
        # Convert throughput from bits per second to megabits per second
        throughput_mbps = sum_data['bits_per_second'] / 1_000_000  # Divide by 1,000,000
        throughputs.append(throughput_mbps)
        retransmissions.append(sum_data.get('retransmits', 0))  # Retransmits might not be present in UDP tests
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Start Time': start_times,
        'End Time': end_times,
        'Duration': durations,
        'Throughput (Mbps)': throughputs,
        'Retransmissions': retransmissions
    })
    
    return df

def filter_zero_throughput(df):
    """Filter out data points where throughput is zero."""
    return df[df['Throughput (Mbps)'] != 0].reset_index(drop=True)

# Load data from both files
df1 = load_iperf_data(file1)
df2 = load_iperf_data(file2)

# Filter out zero throughput data points for plotting throughput
# df1_nonzero = filter_zero_throughput(df1)
# df2_nonzero = filter_zero_throughput(df2)

# Calculate overall throughput and average retransmissions for each dataset
overall_throughput1 = df1['Throughput (Mbps)'].mean()
average_retransmits1 = df1['Retransmissions'].mean()

overall_throughput2 = df2['Throughput (Mbps)'].mean()
average_retransmits2 = df2['Retransmissions'].mean()

print(f"UE 1 - Overall Throughput: {overall_throughput1:.2f} Mbps")
print(f"UE 1 - Average Retransmissions: {average_retransmits1:.2f}")

print(f"\nUE 2 - Overall Throughput: {overall_throughput2:.2f} Mbps")
print(f"UE 2 - Average Retransmissions: {average_retransmits2:.2f}")

# Plotting the throughputs over time (excluding zero throughput data points)
plt.figure(figsize=(12, 6))
plt.plot(df1['Start Time'], df1['Throughput (Mbps)'], label='UE 1 Throughput', marker='o')
plt.plot(df2['Start Time'], df2['Throughput (Mbps)'], label='UE 2 Throughput', marker='x')
plt.title('Throughput Over Time (Excluding Zero Values)')
plt.xlabel('Time (s)')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.grid(True)
plt.show()

# Plotting the retransmissions over time (include all data points)
plt.figure(figsize=(12, 6))
plt.plot(df1['Start Time'], df1['Retransmissions'], label='UE 1 Retransmissions', marker='o')
plt.plot(df2['Start Time'], df2['Retransmissions'], label='UE 2 Retransmissions', marker='x')
plt.title('Retransmissions Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Number of Retransmissions')
plt.legend()
plt.grid(True)
plt.show()


COMMENT: the cwnd calculation is not showing the expected result. 

In [ ]:

name_tx0="prague"


# the csv files generated is of the following format
# timestamp, fd, cwnd, srtt

file_out_tx0_csv = name_tx0+"-ss.csv"

for ue_id in range(1, 3):
    print("Running to generate csv files " + name_tx0)

    ss_tx0_script_processing="""

    f_1={types}; 
    ue_id={ue_id};
    rm -f ${{f_1}}-ss-${{ue_id}}.csv;
    cat ${{f_1}}-ss-${{ue_id}}.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT"> ${{f_1}}-ss-processed-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | awk '{{print $1}}' > ts-${{f_1}}-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | grep -oP '\\bcwnd:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' ' > cwnd-${{f_1}}-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | grep -oP '\\brtt:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' '  | cut -d '/' -f 1   > srtt-${{f_1}}-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | grep -oP '\\bfd=.*?(\s|$)' | awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-${{f_1}}-${{ue_id}}.txt;
    paste ts-${{f_1}}-${{ue_id}}.txt fd-${{f_1}}-${{ue_id}}.txt cwnd-${{f_1}}-${{ue_id}}.txt srtt-${{f_1}}-${{ue_id}}.txt -d ',' > ${{f_1}}-ss-${{ue_id}}.csv;""".format(types=name_tx0, ue_id="ue"+str(ue_id))

    tx.run(ss_tx0_script_processing)

tx.get("prague"+"-ss-ue1.csv")
tx.get("prague"+"-ss-ue2.csv")


In [93]:
import itertools
import json
import pandas as pd

throughput_data = {}  # Initialize the dictionary
srtt_data = {}
cwnd_data= pd.DataFrame()
srtt_data_time= pd.DataFrame()

for ue_id in range(1, 3):
    name_tx0="prague"
    ue_str = "ue"+str(ue_id)

    # Load the JSON output file into a Python object
    with open(f"{name_tx0}-result-{ue_str}.json") as f:
        iperf3_data = json.load(f)

    throughput_data[name_tx0+ue_str] = iperf3_data['end']['sum_received']['bits_per_second'] / (1000000 * 1)  # to convert Mbit

    # Average SRTT for Each Flow
    columns = ['timestamp', 'flow ID', 'cwnd', 'srtt']
    df_f1 = pd.read_csv(f"{name_tx0}-ss-{ue_str}.csv", names=columns)
    
    # Filter out rows with flow ID = 4, they are for the control flows
    df_f1 = df_f1[df_f1['flow ID'] != 4].reset_index(drop=True)

    average_RTT_f1 = df_f1['srtt'].mean()
    
    cwnd_data[name_tx0+ue_str] = df_f1['cwnd']
    srtt_data[name_tx0+ue_str] = average_RTT_f1
    srtt_data_time[name_tx0+ue_str] = df_f1['srtt']

# Save throughput_data to a JSON file
with open('throughput_data.json', 'w') as f:
    json.dump(throughput_data, f)

# Save srtt_data to a JSON file
with open('srtt_data.json', 'w') as f:
    json.dump(srtt_data, f)

cwnd_data.to_csv("consolidated_cwnd_data.csv", index=False)
srtt_data_time.to_csv("time_srtt.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

btl_limit_noecn=int(1000*btl_capacity*base_rtt*n_bdp /8)

print("btl limit no ecn: ", btl_limit_noecn)

# Specify the filename
filename = 'consolidated_cwnd_data.csv'  # Replace with your actual filename

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(filename)

# Extract data for each UE
ue1_data = df['pragueue1']
ue2_data = df['pragueue2']

# Create a figure with two subplots side by side
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# Plot for UE1
axes[0].plot(ue1_data*1448, marker='', color='blue')
axes[0].set_title('Congestion Window (cwnd) Over Time - UE1')
axes[0].set_xlabel('Time Interval')
axes[0].set_ylabel('cwnd (segments)')
axes[0].axhline(y=btl_limit_noecn, color='b', linestyle='--', label=f'Buffer Size')
axes[0].grid(True)

# Plot for UE2
axes[1].plot(ue2_data*1448, marker='', color='orange')
axes[1].set_title('Congestion Window (cwnd) Over Time - UE2')
axes[1].set_xlabel('Time Interval')
axes[1].set_ylabel('cwnd (segments)')
axes[1].axhline(y=btl_limit_noecn, color='b', linestyle='--', label=f'Buffer Size')
axes[1].grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()


In [ ]:
router.close()
router = Connection(
    host='pc487.emulab.net',
    user='PeterYao',
    port=22,
)

router.get("monitor.log")

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Define a function to parse the log file
def parse_qdisc_log(filename):
    timestamps = []
    packet_drops = []
    queue_lengths = []
    ecn_marks = []
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()

            # Extract timestamp
            match_time = re.match(r'^(\d+\.\d+)', line)
            if match_time:
                timestamps.append(float(match_time.group(1)))

            # Extract packet drops, queue length, and ECN mark count for dualpi2 queue
            match_dualpi2 = re.search(r'qdisc dualpi2.*dropped (\d+).*backlog (\d+)b.*ecn_mark (\d+)', line)
            if match_dualpi2:
                packet_drops.append(int(match_dualpi2.group(1)))
                queue_lengths.append(int(match_dualpi2.group(2)))
                ecn_marks.append(int(match_dualpi2.group(3)))

    # Create a DataFrame
    df = pd.DataFrame({
        'Timestamp': timestamps[:len(packet_drops)],  # Align timestamps with the collected data
        'Packet Drops': packet_drops,
        'Queue Length (bytes)': queue_lengths,
        'ECN Marks': ecn_marks
    })
    
    # Normalize timestamps to start from 0
    df['Relative Time (s)'] = df['Timestamp'] - df['Timestamp'].iloc[0]
    
    return df

# Step 2: Define a function to plot packet drops, queue length, and ECN mark count
def plot_qdisc_metrics(df):
    plt.figure(figsize=(12, 8))

    # Plot packet drops
    plt.subplot(3, 1, 1)
    plt.plot(df['Relative Time (s)'], df['Packet Drops'], label="Packet Drops", color='r', marker='o')
    plt.title('Packet Drops Over Time')
    plt.ylabel('Packet Drops')
    plt.grid(True)

    # Plot queue length
    plt.subplot(3, 1, 2)
    plt.plot(df['Relative Time (s)'], df['Queue Length (bytes)'], label="Queue Length", color='b', marker='o')
    plt.title('Queue Length Over Time')
    plt.ylabel('Queue Length (bytes)')
    plt.grid(True)

    # Plot ECN marks
    plt.subplot(3, 1, 3)
    plt.plot(df['Relative Time (s)'], df['ECN Marks'], label="ECN Marks", color='g', marker='o')
    plt.title('ECN Marks Over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('ECN Marks')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Step 3: Read the file and generate plots
filename = 'monitor.log'  # Update the file path as needed
df = parse_qdisc_log(filename)
plot_qdisc_metrics(df)


In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Define a function to parse the log file
def parse_qdisc_log(filename):
    timestamps = []
    packet_drops = []
    queue_lengths = []
    ecn_marks = []
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()

            # Extract timestamp
            match_time = re.match(r'^(\d+\.\d+)', line)
            if match_time:
                timestamps.append(float(match_time.group(1)))

            # Extract packet drops, queue length, and ECN mark count for dualpi2 queue
            match_dualpi2 = re.search(r'qdisc dualpi2.*dropped (\d+).*backlog (\d+)b.*ecn_mark (\d+)', line)
            if match_dualpi2:
                packet_drops.append(int(match_dualpi2.group(1)))
                queue_lengths.append(int(match_dualpi2.group(2)))
                ecn_marks.append(int(match_dualpi2.group(3)))

    # Create a DataFrame
    df = pd.DataFrame({
        'Timestamp': timestamps[:len(packet_drops)],  # Align timestamps with the collected data
        'Packet Drops': packet_drops,
        'Queue Length (bytes)': queue_lengths,
        'ECN Marks': ecn_marks
    })
    
    # Normalize timestamps to start from 0
    df['Relative Time (s)'] = df['Timestamp'] - df['Timestamp'].iloc[0]
    
    return df

# Step 2: Define a function to plot packet drops, queue length, and ECN mark count
def plot_qdisc_metrics(df):
    plt.figure(figsize=(12, 8))

    # Plot packet drops
    plt.subplot(3, 1, 1)
    plt.plot(df['Relative Time (s)'], df['Packet Drops'], label="Packet Drops", color='r', marker='o')
    plt.title('Packet Drops Over Time')
    plt.ylabel('Packet Drops')
    plt.grid(True)

    # Plot queue length
    plt.subplot(3, 1, 2)
    plt.plot(df['Relative Time (s)'], df['Queue Length (bytes)'], label="Queue Length", color='b', marker='o')
    plt.title('Queue Length Over Time')
    plt.ylabel('Queue Length (bytes)')
    plt.grid(True)

    # Plot ECN marks
    plt.subplot(3, 1, 3)
    plt.plot(df['Relative Time (s)'], df['ECN Marks'], label="ECN Marks", color='g', marker='o')
    plt.title('ECN Marks Over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('ECN Marks')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Step 3: Read the file and generate plots
filename = 'monitor.log'  # Replace this with the actual path to your file
df = parse_qdisc_log(filename)
plot_qdisc_metrics(df)


In [ ]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# File path to the uploaded CSV file
file_path = 'time_srtt.csv'

# Reading the CSV file
df = pd.read_csv(file_path)

# Calculate the average SRTT for each UE
avg_ue1 = df['pragueue1'].mean()
avg_ue2 = df['pragueue2'].mean()

# Plot the SRTT changes over time for both UE1 and UE2
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['pragueue1'], label='pragueue1', marker='o')
plt.plot(df.index, df['pragueue2'], label='pragueue2', marker='s')

# Plot average lines
plt.axhline(y=avg_ue1, color='blue', linestyle='--', linewidth=1, label=f'Average UE1 ({avg_ue1:.2f} ms)')
plt.axhline(y=avg_ue2, color='green', linestyle='--', linewidth=1, label=f'Average UE2 ({avg_ue2:.2f} ms)')

# Adding labels and title
plt.xlabel('Time (Index)')
plt.ylabel('SRTT (ms)')
plt.title('SRTT Changes Over Time for cubic_ecn_noneue1 and cubic_ecn_noneue2')

# Adding a legend and grid
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Display the plot
plt.tight_layout()
plt.show()

# Print the average SRTT for each UE
print(f"Average SRTT for cubic_ecn_noneue1: {avg_ue1:.3f} ms")
print(f"Average SRTT for cubic_ecn_noneue2: {avg_ue2:.3f} ms")


: 